# How to structure a working code: a small example

This simple script plots the info about the students of a classroom, the real students of the course itself.

In particular, we limit to the info about the Departments (cds) they belong to.

## Structure

To make the code simple to use and read, let's divide into three scripts having different scopes

1. classroom.py : load the students' database and estract info by using the id_code (matricola) or the surname
1. customplots.py : prepare the plot as pie or bar to show the info graphically
1. stats_on_cds.py : print the info about the cds of the students

## classroom.py

In [2]:
"""
Script to load info
of students from a cvs database
"""
import os
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import webbrowser

def cds_polito(fname):
    """
    get the Names of the Departments at polito
    given as short name, long name
    """
    cds = dict()
    with open(fname) as f:
        data = f.readlines()
    for row in data:
        # Split only using the first comma
        s, l = row.split(",", 1)
        # Populate the dict
        cds[s] = l.strip() # Clean the spaces
    return cds

class Classroom:
    """
    class to load a csv file
    containing a table of students
    
    It is a combination of semi-private methods like
    _get_student_info_by_*
    and a public one 
        get_student_info(tag)
    which accepts a tag as a number (id_code, matricola)
    and as a string (surname)
    """
    def __init__(self, mainDir, csv_filename, cds_filename):
        filename = os.path.join(mainDir, csv_filename)
        self.students = pd.read_csv(filename, index_col=0)
        self.surnames = list(self.students['COGNOME'])
        self._id_codes = list(self.students.index)
        self._cds_polito = cds_polito(os.path.join(mainDir, cds_filename))
        
    def _get_student_info_by_surname(self, surname):
        """
        returns a pandas dataframe
        with the students' infos
        based on the surname
        """
        # Surname are saves as Capital, so let's transform
        surname = surname.upper()
        if surname in self.surnames:
            # extract the student as a dataframe of pandas
            df = self.students[self.students['COGNOME'] == surname]
            return df
        else:
            print("Surname %s not in the list" % surname)
            return None
        # Can be done with try ... except ...

    def _get_student_info_by_id(self, id_code):
        """
        returns a pandas dataframe
        with the students' info
        based on id_code (Matricola)
        """
        return self.students.loc[id_code]

    def _get_student_info(self, tag=None):
        if not tag:
            print("Please give a id code or his/her surname")
            return
        else:
            return self._get_df_from_tag(tag)

    def get_student_info(self, tag=None):
        """
        Why two methods with the same name
        just one private?
        If we return a dataframe,
        pandas will plot it, 
        and this is what we do not want to
        """
        df = self._get_student_info(tag)
        if df is not None:
            print(df.to_string())
        return

    def _get_df_from_tag(self, tag):
        """
        get the pandas dataframe
        corresponding a tag (id_code or surname)

        tag: int or string (also as 'ALL')
        """
        if type(tag) == int and tag in self._id_codes:
            df = self._get_student_info_by_id(tag)
        elif type(tag) == str:
            tag = tag.upper() 
            if tag in self.surnames:
                df = self._get_student_info_by_surname(tag)
            elif tag == 'ALL':
                df = self.students
            else:
                df = None
                print("%s surname does not exist" % tag)
        else:
            print("There is a problem, check your input")
            df = None
        return df


    def get_students_cds(self, cds):
        cds_short = cds[:3]
        cds_long = self._cds_polito[cds_short]
        locations = self.students['CDS STUDENTE'] == cds
        st = sum(locations)
        # Can we adjust plural and singular?
        if st == 1:
            verb, stud = "is", ""
        else:
            verb, stud = "are", "s"
        print("There %s %i student%s from %s" % (verb, st, stud, cds_long))
        return st

    def get_student_photo(self, tag=None):
        """
        This method unfortunately works
        only if connected to the polito network
        """
        df = self._get_student_info(tag)
        code = df.index.tolist()[0]
        try: 
            url = "https://didattica.polito.it/pls/portal30/sviluppo.foto_studente?p_matricola=%s" % code
            response = requests.get(url)
            cnt = response.content
            img = Image.open(BytesIO(cnt))
        except OSError:
            cnt = cnt.decode("utf-8")
            if 'ACCESS DENIED' in cnt:
                 webbrowser.open(url)
        except:
            print("Impossible to connect to the Polito site")

    @property
    def cds(self):
        return set(self.students['CDS STUDENTE'])

if __name__ == "__main__":
    mainDir = "/home/gf/src/Python/Python-in-the-lab/project"
    csv_file = "01RONKG_2017.csv"
    cds_file ='phds.txt'
    filename = os.path.join(mainDir, csv_file)
    cl = Classroom(mainDir, csv_file, cds_file)
    for c in sorted(cl.cds):
        cl.get_students_cds(c)

There are 3 students from Aerospace Engineering
There are 3 students from Bioengineering and Medical-Surgical sciences
There are 2 students from Chemical Engineering
There are 14 students from Electrical, Electronics and Communications Engineering
There are 4 students from Energetics
There are 3 students from Physics
There is 1 student from Civil and Environmental Engineering
There are 15 students from Computer and Control Engineering
There are 3 students from Mechanical Engineering
There are 4 students from Metrology
There are 2 students from Materials Science and Technology
There is 1 student from Urban and Regional Development


## customplots.py

In [40]:
"""
custom plots of pie and bar
"""

import numpy as np


class CustomPlots:
    def __init__(self, ax):
        self.ax = ax

    def plot_xy(self, x, y, marker='o', color='r'):
        self.ax.plot(x, y, mk=marker, cl=color)

    def plot_pie(self, size, labels, explode=None, autopct='%1.1f%%',
        shadow=True, startangle=90):
        patches, texts, autotext = self.ax.pie(size, labels=labels, 
            explode=explode, autopct=autopct, shadow=shadow, 
            startangle=startangle)
        self.ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        [t.set_fontsize(22) for t in texts]
    
    def plot_bar(self, size, labels, width=0.4):
        x = np.arange(1, len(labels)+1)
        self.ax.bar(x, size, width)
        self.ax.set_ylabel('Students')
        self.ax.set_xticks(x + width / 2)
        self.ax.set_xticklabels(tuple(labels))


if __name__ == "__main__":
    import matplotlib.pyplot as plt
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
    sizes = [15, 30, 45, 10]
    explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')
    fig1, ax1 = plt.subplots()
    cp = CustomPlots(ax1)
    cp.plot_pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
    plt.show()


## stats_on_cds.py

In [42]:
"""
This is a demo 
for the Python-in-the-lab course
to illustrate how a simple project
can be structured

The final goal is to plot the different Departments
to which the students of the Python course
belong to.
This is done calling the class
Plot_cds_stats twice,
using pie and bar plots
"""
import os
import numpy as np
import matplotlib.pyplot as plt
# The lines below are adapted to work inside a notebook, 
# with the two files in the cells above.
try:
    import classroom as cl 
    import customplots as cp
except:
    import __main__ as cl
    import __main__ as cp

class Plot_cds_stats:
    """
    class to print the data about the students
    """
    def __init__(self, mainDir, csv_filename, cds_filename):
        # Call the Classroom class from classroom.py
        my_class = cl.Classroom(mainDir, csv_filename, cds_filename)
        # cds: codes of Polito Departments
        cds = sorted(my_class.cds)
        self.students = [my_class.get_students_cds(c) for c in cds]
        # Only the first 3 elements of the Departments code are needed
        self.labels = [c[:3] for c in cds]
    
    def pie(self, ax, explode=0.03):
        """
        plot a pie plot using axis ax
        """
        this_plot = cp.CustomPlots(ax)
        if explode is not None:
            if type(explode)  == float:
                # Let's explode all the pies with the same value
                explode = explode * np.ones(len(self.students))
        # Draw the plot
        this_plot.plot_pie(size=self.students, labels=self.labels, explode=explode)
    
    def bar(self, ax):
        """
        plot a bar plot using axis ax
        """
        this_plot = cp.CustomPlots(ax)
        # Draw the plot
        this_plot.plot_bar(size=self.students, labels=self.labels)


if __name__ == "__main__":
    # The plot is setup here, and not in the classes 
    fig, ax = plt.subplots(1,2, figsize=(18,8))
    # Load the file with the students' info
    mainDir = "/home/gf/src/Python/Python-in-the-lab/project"
    csv_file = "01RONKG_2017.csv"
    cds_file = "phds.txt"
    # Initialize the class to plot
    plot = Plot_cds_stats(mainDir, csv_file, cds_file)
    plot.pie(ax[0], explode=0.03)
    plot.bar(ax[1])
    plt.show()

There are 3 students from Aerospace Engineering
There are 3 students from Bioengineering and Medical-Surgical sciences
There are 2 students from Chemical Engineering
There are 14 students from Electrical, Electronics and Communications Engineering
There are 4 students from Energetics
There are 3 students from Physics
There is 1 student from Civil and Environmental Engineering
There are 15 students from Computer and Control Engineering
There are 3 students from Mechanical Engineering
There are 4 students from Metrology
There are 2 students from Materials Science and Technology
There is 1 student from Urban and Regional Development
